In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

In [2]:
# Tensorboard magic, remember to set your host and port, then forward them
# You can set reload data under settings to see data update as you run your model
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard --host=10.128.10.14 --port=8008

# Data Preparation

In [3]:
# defining hyperparameters
batch_size = 32

def apply_image_transformation(transformation_type="standard", *args, **kwargs):
    '''
    Apply various image transformations based on the provided transformation_type.

    Args:
    transformation_type (str): The type of transformation to apply. Supported types are 'standard', 'resize', and 'channel'.
    *args: Additional arguments based on the transformation type.
    **kwargs: Additional keyword arguments for normalization parameters.

    Returns:
    transform: A composition of transformations to be applied to the input images.

    Raises:
    ValueError: If an unsupported transformation type is provided.

    '''
    if transformation_type == "standard":
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(0.5, 0.5)
        ])
        return transform

    elif transformation_type == "resize":
        transform = transforms.Compose([
            transforms.Resize(args[0]),
            transforms.ToTensor(),
            transforms.Normalize(kwargs['mean'], kwargs['std'])
        ])
        return transform

    elif transformation_type == "channel":
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.Resize((224, 224)),
            transforms.Grayscale(args[0]),
            transforms.ToTensor(),
            transforms.Normalize(mean=kwargs['mean'], std=kwargs['std'])
        ])
        return transform

    else:
        raise ValueError("Invalid transformation type. Supported types are 'normal', 'resize', and 'grayscale'.")

def create_loader(transform, batch_size=32):
    '''
    Create data loaders for training and testing using the provided transformation.

    Args:
    transform: The transformation to be applied to the dataset.
    batch_size: The batch size of 

    Returns:
    train_loader: DataLoader for the training dataset.
    test_loader: DataLoader for the testing dataset.

    This function imports the FashionMNIST dataset from the torchvision library and applies the provided transformation to the dataset. It then creates data loaders for both the training and testing datasets, considering the specified transformation and other default parameters such as the number of workers and batch size.
    '''
    # importing training and test sets from torchvision
    train_dataset = torchvision.datasets.FashionMNIST("./data", download=True, train=True, transform=transform)
    test_dataset = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=transform)
    
    # creating dataloaders
    train_loader = DataLoader(train_dataset, shuffle=True, num_workers=2, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, shuffle=False, num_workers=2, batch_size=batch_size)

    return train_loader, test_loader

In [4]:
transform = apply_image_transformation('standard')
basic_train_loader, basic_test_loader = create_loader(transform)

transform = apply_image_transformation('standard')
vgg_train_loader, vgg_test_loader = create_loader(transform)

transform = apply_image_transformation('resize', 224, mean=(0.1307,), std=(0.3081,))
vgg_resize_train_loader, vgg_resize_test_loader = create_loader(transform)

transform = apply_image_transformation('channel', 3, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
vgg_channel_train_loader, vgg_channel_test_loader = create_loader(transform)

# Training

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

def train(model, criterion, optimizer, num_epochs, train_loader, test_loader, writer=None):
  total_time_taken = 0.0
  # Train the model
  for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    correct = 0
    for i, data in enumerate(tqdm(train_loader)):
      # Get the inputs and labels
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      # Zero the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(inputs)

      # Compute the loss
      loss = criterion(outputs, labels)
      train_loss += loss.item()

      # Backward pass and optimize
      loss.backward()
      optimizer.step()

      # Compute correct predictions
      pred = outputs.argmax(dim=1, keepdim=True)
      correct += pred.eq(labels.view_as(pred)).sum().item()

      # Log training loss per 200 mini batches using TensorBoard
      if writer:
        if i % 200 == 199:
          writer.add_scalar('Train Loss', train_loss/200, epoch * len(train_loader) + i)

    total_time_taken += time.time() - start_time

    # Compute train accuracy
    train_accuracy = 100. * correct / len(train_loader.dataset)

    # Log training loss per 200 mini batches using TensorBoard
    if writer:
      writer.add_scalar('Train Accuracy', train_accuracy, epoch)

    model.eval()
    test_loss = 0.0
    correct = 0
    test_acc = []
    with torch.no_grad():
      for i, data in enumerate(test_loader):
        # Get the inputs and labels
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Compute correct predictions
        pred = outputs.argmax(dim=1, keepdim=True)
        correct += pred.eq(labels.view_as(pred)).sum().item()

        # Log training loss per 200 mini batches using TensorBoard
        if writer:
          if i % 200 == 199:
            writer.add_scalar('Test Loss', train_loss/200, epoch * len(test_loader) + i)

    # Compute test accuracy
    test_accuracy = 100. * correct / len(test_loader.dataset)
    test_acc.append(test_accuracy)

    # Log training loss per 200 mini batches using TensorBoard
    if writer:
      writer.add_scalar('Test Accuracy', test_accuracy, epoch)

    # Print statistics
    print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%')

  print(f'\n\nTotal Time Elapsed: {total_time_taken} s')

## Batch Normalization

Rather than using a simple convolution layer, each convolution layer is transformed into a 3 layer architecture
- Convolution Layer
- Batch Normalization Layer
- Activation Layer (LeakyReLU)

In [6]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, stride=1):
        super(BasicConv2d, self).__init__()
        self.basicconv2d = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        return self.basicconv2d(x)

## Basic CNN

In [7]:
class BasicCNN(nn.Module):
  def __init__(self):
    super(BasicCNN, self).__init__()
    self.conv1 = BasicConv2d(1, 32, 3) # 32 1x3x3 filters with stride 1, pad 0
    '''
    Output size = (28 - 3 + 2*0)/1 + 1 = 26
    Output volume = 32x26x26
    '''
    self.pool = nn.MaxPool2d(2, 2) # 2x2 filter with stride 2
    '''
    Output size = (26 - 2)/2 + 1 = 13
    Output volume = 32x13x13
    '''
    self.fc1 = nn.Linear(32 * 13 * 13, 100)
    self.fc2 = nn.Linear(100, 10)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = self.pool(x)
    # Flatten the output of the last convolutional layer
    x = x.view(-1, 32 * 13 * 13)
    # Apply the fully connected layers with ReLU activation
    x = torch.relu(self.fc1(x))
    # Apply the last fully connected layer with softmax activation
    x = self.fc2(x)

    return x

In [8]:
# defining hyperparameters
learning_rate = 0.001
num_epochs = 10

BasicCNNModel = BasicCNN().to(device)
writer = SummaryWriter('logs/tensorboard/Basic/BasicModel')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(BasicCNNModel.parameters(), lr=learning_rate)

train(BasicCNNModel, criterion, optimizer, num_epochs, basic_train_loader, basic_test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.3786, Train Accuracy: 86.53%, Test Loss: 0.3063, Test Accuracy: 88.79%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 2, Train Loss: 0.2656, Train Accuracy: 90.21%, Test Loss: 0.2809, Test Accuracy: 89.74%


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8, Train Loss: 0.1160, Train Accuracy: 95.68%, Test Loss: 0.3134, Test Accuracy: 91.00%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9, Train Loss: 0.0994, Train Accuracy: 96.23%, Test Loss: 0.3406, Test Accuracy: 91.06%


100%|██████████| 1875/1875 [00:09<00:00, 202.88it/s]


Epoch 10, Train Loss: 0.0897, Train Accuracy: 96.65%, Test Loss: 0.3390, Test Accuracy: 90.37%


Total Time Elapsed: 93.64534449577332 s


## VGG Architecture
The VGG architecture proposed in the original paper worked on an input image dimension of 224x224x3. Since the Fashion MNIST dataset are of 28x28x1 dimension, either the image has to be rescaled, or the architecture has to be modified to better fit our use case.

### Original VGG16 Architecture
For this implementation, we utilize the original VGG16 architecture with a few key modifications to accommodate the Fashion MNIST dataset. Initially, we resize the input image dimensions to 224x224x1 to align with the VGG model's input requirements. Additionally, we introduce small dropout layers after each fully connected layer. This measure helps prevent overfitting, considering that the original VGG model was primarily designed for the ImageNet dataset, which contains a significantly larger volume of images. 
s.
![](https://media.geeksforgeeks.org/wp-content/uploads/20200219152207/new41.jpg)

### Modified VGG Architecture
Propsed architecture 1:
- 2x Convulution + ReLU (28 x 28 x 64)
- Max Pooling (14 x 14 x 64)
- 2x Convolution + ReLU (14 x 14 x 128)
- Max Pooling (7 x 7 x 128)
- 3x Convolution + ReLU (7 x 7 x 256)
- Max Pooling (3 x 3 x 256)
- 3x Convolution + ReLU (3 x 3 x 512)
- Max Pooling (1 x 1 x 512)

Propsed architecture 1:
- 2x Convulution + ReLU (28 x 28 x 64)
- Max Pooling (14 x 14 x 64)
- 2x Convolution + ReLU (14 x 14 x 128)
- Max Pooling (7 x 7 x 128)
- 3x Convolution + ReLU (7 x 7 x 256)
- Max Pooling (3 x 3 x 256)

Propsed architecture 1:
- 2x Convulution + ReLU (28 x 28 x 64)
- Max Pooling (14 x 14 x 64)
- 2x Convolution + ReLU (14 x 14 x 128)
- Max Pooling (7 x 7 x 128)

Fully connected layer for all three architecture will follow a similar 3-FC layer as implemented in the original VGG16. However, the output dimension each layer have been scaled down accordingly to the nearest power 2.

Example:
- Proposed architecture 1: Scaled down from 7x7x512 to 1x1x512, by a factor of 49 => 4096/49 = 84 => 64

In [9]:
class VGG16(nn.Module):
  def __init__(self):
    super(VGG16, self).__init__()
    self.conv1_1 = BasicConv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
    self.conv1_2 = BasicConv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

    self.conv2_1 = BasicConv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv2_2 = BasicConv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

    self.conv3_1 = BasicConv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.conv3_2 = BasicConv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
    self.conv3_3 = BasicConv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

    self.conv4_1 = BasicConv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
    self.conv4_2 = BasicConv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv4_3 = BasicConv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.conv5_1 = BasicConv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv5_2 = BasicConv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv5_3 = BasicConv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = nn.Linear(7 * 7 * 512, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 10)

  def forward(self, x):
    x = torch.relu(self.conv1_1(x))
    x = torch.relu(self.conv1_2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv2_1(x))
    x = torch.relu(self.conv2_2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv3_1(x))
    x = torch.relu(self.conv3_2(x))
    x = torch.relu(self.conv3_3(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv4_1(x))
    x = torch.relu(self.conv4_2(x))
    x = torch.relu(self.conv4_3(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv5_1(x))
    x = torch.relu(self.conv5_2(x))
    x = torch.relu(self.conv5_3(x))
    x = self.maxpool(x)
    x = x.view(-1, 7 * 7 * 512)
    x = torch.relu(self.fc1(x))
    x = nn.functional.dropout(x, 0.2)
    x = torch.relu(self.fc2(x))
    x = nn.functional.dropout(x, 0.2)
    x = self.fc3(x)

    return x

In [10]:
# defining hyperparameters
learning_rate = 0.001
num_epochs = 10

VGGModel = VGG16().to(device)
writer = SummaryWriter('logs/tensorboard/VGG/VGG16')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(VGGModel.parameters(), lr=learning_rate)

train(VGGModel, criterion, optimizer, num_epochs, vgg_resize_train_loader, vgg_resize_test_loader, writer)

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.8073, Train Accuracy: 74.27%, Test Loss: 0.5228, Test Accuracy: 81.33%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 2, Train Loss: 0.4148, Train Accuracy: 85.03%, Test Loss: 0.3686, Test Accuracy: 86.45%


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8, Train Loss: 0.1859, Train Accuracy: 93.34%, Test Loss: 0.2406, Test Accuracy: 91.85%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9, Train Loss: 0.1733, Train Accuracy: 93.88%, Test Loss: 0.2317, Test Accuracy: 92.22%


100%|██████████| 1875/1875 [06:17<00:00,  4.97it/s]


Epoch 10, Train Loss: 0.1542, Train Accuracy: 94.55%, Test Loss: 0.2214, Test Accuracy: 92.29%


Total Time Elapsed: 3773.766048669815 s


In [11]:
class ModifiedVGG_1(nn.Module):
  def __init__(self):
    super(ModifiedVGG_1, self).__init__()
    self.conv1_1 = BasicConv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
    self.conv1_2 = BasicConv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

    self.conv2_1 = BasicConv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv2_2 = BasicConv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

    self.conv3_1 = BasicConv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.conv3_2 = BasicConv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
    self.conv3_3 = BasicConv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

    self.conv4_1 = BasicConv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
    self.conv4_2 = BasicConv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv4_3 = BasicConv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = nn.Linear(1 * 1 * 512, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = torch.relu(self.conv1_1(x))
    x = torch.relu(self.conv1_2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv2_1(x))
    x = torch.relu(self.conv2_2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv3_1(x))
    x = torch.relu(self.conv3_2(x))
    x = torch.relu(self.conv3_3(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv4_1(x))
    x = torch.relu(self.conv4_2(x))
    x = torch.relu(self.conv4_3(x))
    x = self.maxpool(x)
    x = x.view(-1, 1 * 1 * 512)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [12]:
# defining hyperparameters
learning_rate = 0.001
num_epochs = 10

ModifiedVGGModel_1 = ModifiedVGG_1().to(device)
writer = SummaryWriter('logs/tensorboard/VGG/ModifiedVGG_1')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ModifiedVGGModel_1.parameters(), lr=learning_rate)

train(ModifiedVGGModel_1, criterion, optimizer, num_epochs, vgg_train_loader, vgg_test_loader, writer)

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.4438, Train Accuracy: 83.88%, Test Loss: 0.2927, Test Accuracy: 89.55%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 2, Train Loss: 0.2724, Train Accuracy: 90.34%, Test Loss: 0.3176, Test Accuracy: 88.49%


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8, Train Loss: 0.1179, Train Accuracy: 95.79%, Test Loss: 0.2233, Test Accuracy: 92.88%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9, Train Loss: 0.0999, Train Accuracy: 96.42%, Test Loss: 0.2167, Test Accuracy: 93.06%


100%|██████████| 1875/1875 [00:19<00:00, 98.09it/s]


Epoch 10, Train Loss: 0.0873, Train Accuracy: 96.92%, Test Loss: 0.2436, Test Accuracy: 92.82%


Total Time Elapsed: 192.8491780757904 s


In [13]:
class ModifiedVGG_2(nn.Module):
  def __init__(self):
    super(ModifiedVGG_2, self).__init__()
    self.conv1_1 = BasicConv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
    self.conv1_2 = BasicConv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

    self.conv2_1 = BasicConv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv2_2 = BasicConv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

    self.conv3_1 = BasicConv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.conv3_2 = BasicConv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
    self.conv3_3 = BasicConv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = nn.Linear(3 * 3 * 256, 256)
    self.fc2 = nn.Linear(256, 256)
    self.fc3 = nn.Linear(256, 10)

  def forward(self, x):
    x = torch.relu(self.conv1_1(x))
    x = torch.relu(self.conv1_2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv2_1(x))
    x = torch.relu(self.conv2_2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv3_1(x))
    x = torch.relu(self.conv3_2(x))
    x = torch.relu(self.conv3_3(x))
    x = self.maxpool(x)
    x = x.view(-1, 3 * 3 * 256)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [14]:
# defining hyperparameters
learning_rate = 0.001
num_epochs = 10

ModifiedVGGModel_2 = ModifiedVGG_2().to(device)
writer = SummaryWriter('logs/tensorboard/VGG/ModifiedVGG_2')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ModifiedVGGModel_2.parameters(), lr=learning_rate)

train(ModifiedVGGModel_2, criterion, optimizer, num_epochs, vgg_train_loader, vgg_test_loader, writer)

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.3900, Train Accuracy: 85.75%, Test Loss: 0.3224, Test Accuracy: 88.54%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 2, Train Loss: 0.2532, Train Accuracy: 90.67%, Test Loss: 0.2738, Test Accuracy: 90.15%


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8, Train Loss: 0.1013, Train Accuracy: 96.23%, Test Loss: 0.2325, Test Accuracy: 93.26%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9, Train Loss: 0.0867, Train Accuracy: 96.81%, Test Loss: 0.2502, Test Accuracy: 92.22%


100%|██████████| 1875/1875 [00:15<00:00, 122.81it/s]


Epoch 10, Train Loss: 0.0729, Train Accuracy: 97.37%, Test Loss: 0.2629, Test Accuracy: 93.18%


Total Time Elapsed: 153.30515146255493 s


In [15]:
class ModifiedVGG_3(nn.Module):
  def __init__(self):
    super(ModifiedVGG_3, self).__init__()
    self.conv1_1 = BasicConv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
    self.conv1_2 = BasicConv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

    self.conv2_1 = BasicConv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv2_2 = BasicConv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = nn.Linear(7 * 7 * 128, 1024)
    self.fc2 = nn.Linear(1024, 1024)
    self.fc3 = nn.Linear(1024, 10)

  def forward(self, x):
    x = torch.relu(self.conv1_1(x))
    x = torch.relu(self.conv1_2(x))
    x = self.maxpool(x)
    x = torch.relu(self.conv2_1(x))
    x = torch.relu(self.conv2_2(x))
    x = self.maxpool(x)
    x = x.view(-1, 7 * 7 * 128)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [16]:
# defining hyperparameters
learning_rate = 0.001
num_epochs = 10

ModifiedVGGModel_3 = ModifiedVGG_3().to(device)
writer = SummaryWriter('logs/tensorboard/VGG/ModifiedVGG_3')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ModifiedVGGModel_3.parameters(), lr=learning_rate)

train(ModifiedVGGModel_3, criterion, optimizer, num_epochs, vgg_train_loader, vgg_test_loader, writer)

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.4070, Train Accuracy: 85.28%, Test Loss: 0.2780, Test Accuracy: 90.01%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 2, Train Loss: 0.2472, Train Accuracy: 91.08%, Test Loss: 0.2366, Test Accuracy: 91.47%


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8, Train Loss: 0.0817, Train Accuracy: 96.99%, Test Loss: 0.2807, Test Accuracy: 92.59%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9, Train Loss: 0.0659, Train Accuracy: 97.60%, Test Loss: 0.3046, Test Accuracy: 92.77%


100%|██████████| 1875/1875 [00:14<00:00, 131.88it/s]


Epoch 10, Train Loss: 0.0569, Train Accuracy: 97.97%, Test Loss: 0.2991, Test Accuracy: 92.85%


Total Time Elapsed: 128.95099425315857 s


# ResNet

# Inceptionv3

---
# Transfer Learning

## VGG16

In [7]:
PretrainedVGGModel = torch.hub.load('pytorch/vision:v0.7.0', 'vgg16', pretrained=True).to(device)
writer = SummaryWriter('logs/tensorboard/VGG/PretrainedVGG')

# defining hyperparameters
learning_rate = 0.001
num_epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(PretrainedVGGModel.parameters(), lr=learning_rate)

train(PretrainedVGGModel, criterion, optimizer, num_epochs, vgg_channel_train_loader, vgg_channel_test_loader)

Using cache found in /home/UG/chua0994/.cache/torch/hub/pytorch_vision_v0.7.0
  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.6096, Train Accuracy: 80.42%, Test Loss: 0.3625, Test Accuracy: 86.54%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 2, Train Loss: 0.3564, Train Accuracy: 87.19%, Test Loss: 0.3189, Test Accuracy: 87.79%


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8, Train Loss: 0.2703, Train Accuracy: 90.08%, Test Loss: 0.2730, Test Accuracy: 90.21%


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9, Train Loss: 0.2487, Train Accuracy: 90.89%, Test Loss: 0.2920, Test Accuracy: 89.53%


100%|██████████| 1875/1875 [05:49<00:00,  5.36it/s]


Epoch 10, Train Loss: 0.2446, Train Accuracy: 90.99%, Test Loss: 0.2705, Test Accuracy: 90.38%


Total Time Elapsed: 3498.5282509326935 s
